In [10]:
from pymongo import MongoClient
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

db_user = config['DEFAULT']['DB_USER']
db_name = config['DEFAULT']['DB_NAME']
db_pass = config['DEFAULT']['DB_PASS']

db_url = "mongodb://" + db_user + ":" + db_pass + "@ds237669.mlab.com:37669/" + db_name

client = MongoClient(db_url)
db = client[db_name]

In [11]:
class User(object):    
    def __init__(self, **entries):
        self.__dict__.update(entries)        
        
class Rating(object):    
    def __init__(self, **entries):
        self.__dict__.update(entries)
    
    def get_rating(self):
        return self.rating/2.0

#### Exporting csv of training data

In [12]:
db_users = db.users.find({})

data_list = []

for u in db_users:
    args = u
    user = User(**args)
    m_ratings = db.rateds.find({'_user': user._id})
    
    for r in m_ratings:
        args = r
        rating = Rating(**args)
        
        # TODO: user app.core.util_constants.TrainingDataConstants
        new_row = {
            'user_id': user._id,
            'item_id': rating._media,
            'rating': rating.get_rating()}
        
        data_list.append(new_row)

In [4]:
import pandas as pd
import os
    
df = pd.DataFrame(columns=('user_id', 'item_id', 'rating'))
df = pd.DataFrame(data_list)

data_dir = "../data/"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

df.to_csv(data_dir + 'kitso_data.csv', index=False, sep=";")

#### Exporting csv with kitso movies

In [5]:
import requests

movie_list = []
already_requested_movie = []

# TODO: use as constants defined in app
kitso_api_url = "http://kitso.herokuapp.com/api"
MOVIE_ENDPOINT = "/movie/"

for d in data_list:
    movie_id = d['item_id']
    
    if movie_id in already_requested_movie:
        continue
    
    url = kitso_api_url + MOVIE_ENDPOINT + str(movie_id)
    resp = requests.get(url)
    
    # TODO: use status code int defined in app constants
    if resp.status_code >= 200 and resp.status_code < 300:        
        resp_dict = resp.json()
        
        release_year = resp_dict["release_date"].split("-")[0]
        title_and_year = resp_dict["title"] + " (" + release_year + ")"
        
        # TODO: user app.core.util_constants.ItemsDataConstants
        new_row = {'kitso_id': resp_dict["_id"],
                   'imdb_id': resp_dict["imdb_id"],
                   'tmdb_id': resp_dict["id"],
                   'original_title': resp_dict["original_title"],
                   'title': resp_dict["title"],
                   '_t': resp_dict["__t"],
                   'poster_path': resp_dict["poster_path"],
                   'title_year': title_and_year}
        
        movie_list.append(new_row)
        already_requested_movie.append(movie_id)

In [6]:
df = pd.DataFrame(columns=('kitso_id', 'imdb_id', 'tmdb_id', 'original_title', 'title', '_t', 'poster_path'))
df = pd.DataFrame(movie_list)
df.to_csv(data_dir + 'kitso_movies_data.csv', index=False, sep=";")